In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [2]:
df20 = df20.apply(pd.to_numeric, errors = 'coerce')
df20.describe().iloc[:, :15]
# 없는 Column = date, codesum, depart, sunrise, sunset, station_nbr

,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure
count,1.042290e+05,104007.000000,103785.000000,103452.000000,0.0,103563.000000,102120.000000,103452.000000,103452.000000,0.0,0.0,0.0,101565.0,104118.000000,103341.000000
mean,1.368430e+18,79.162220,57.444920,68.642704,NaN,51.517685,58.828261,5.388412,9.031116,NaN,NaN,NaN,0.0,0.061439,29.289689
std,2.613970e+16,16.824979,16.508153,16.311362,NaN,16.064089,13.767896,8.896285,9.465144,NaN,NaN,NaN,0.0,0.267002,0.163950
min,1.325376e+18,25.000000,14.000000,17.000000,NaN,4.000000,19.000000,0.000000,0.000000,NaN,NaN,NaN,0.0,0.000000,28.780000
25%,1.345594e+18,68.000000,43.000000,55.000000,NaN,38.000000,47.000000,0.000000,0.000000,NaN,NaN,NaN,0.0,0.000000,29.180000
50%,1.367107e+18,81.000000,60.000000,71.000000,NaN,56.000000,62.000000,0.000000,6.000000,NaN,NaN,NaN,0.0,0.000000,29.260000
75%,1.391731e+18,93.000000,73.000000,83.000000,NaN,66.000000,72.000000,10.000000,18.000000,NaN,NaN,NaN,0.0,0.005000,29.380000
max,1.414714e+18,108.000000,83.000000,94.000000,NaN,72.000000,76.000000,48.000000,29.000000,NaN,NaN,NaN,0.0,3.340000,29.900000


In [3]:
df20['store_nbr'].unique()

array([17], dtype=int64)

In [4]:
df20_drop_columns = ['date', 'codesum', 'depart', 'sunrise', 'sunset', 'station_nbr']
df20 = df20.drop(columns = df20_drop_columns)

In [5]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df20_columns = df20.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df20_columns:
    if (i == 'resultdir'):
        df20[i].fillna(df20[i].mode()[0], inplace=True)
        print(df20[i].mode()[0])
    else:
        df20[i].fillna(df20[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df20['relative_humility'] = 100*(np.exp((20.625*((df20['dewpoint']-32)/1.8))/(243.04+((df20['dewpoint']-32)/1.8)))/np.exp((20.625*((df20['tavg']-32)/1.8))/(243.04+((df20['tavg']-32)/1.8))))

# 체감온도 계산
df20["windchill"] = 35.74 + 0.6220*df20["tavg"] - 35.75*(df20["avgspeed"]**0.20) + 0.4275*df20["tavg"]*(df20["avgspeed"]**0.20)

df20 = df20[df20['units'] != 0]

17.0


In [6]:
# 'np.log1p(units) ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesume + \
# snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + nothing + rain + other + \
# store_nbr + station_nbr + item_nbr'

model_df20 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
resultspeed + sealevel + snowfall + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df20)
result_df20 = model_df20.fit()

print(result_df20.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                     860.5
Date:                Thu, 05 Jul 2018   Prob (F-statistic):               0.00
Time:                        21:58:48   Log-Likelihood:                -1446.0
No. Observations:                2627   AIC:                             2942.
Df Residuals:                    2602   BIC:                             3089.
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(nothing)[0]        -0.0203      0.02

In [7]:
model_df20 = sm.OLS.from_formula('np.log1p(units) ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + \
scale(preciptotal) + scale(resultspeed) + scale(sealevel) + scale(snowfall)+ scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + \
C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df20)
result_df20 = model_df20.fit()

print(result_df20.summary())

ValueError: zero-size array to reduction operation maximum which has no identity

In [8]:
anova_result_df20 = sm.stats.anova_lm(result_df20).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df20[anova_result_df20['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
relative_humility,1.0,1.362549,1.362549,7.666077,5.666743e-03
heat,1.0,3.457570,3.457570,19.453251,1.072819e-05
C(week7),6.0,28.656147,4.776025,26.871240,3.184809e-31
C(item_nbr),3.0,3632.312731,1210.770910,6812.133242,0.000000e+00
C(nothing),2.0,46574.695900,23287.347950,131021.083969,0.000000e+00


In [9]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df20.values, i) for i in range(df20.shape[1])]
vif["features"] = df20.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# scale(relative_humility) + C(week7) + C(item_nbr) 

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1386: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1386: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,VIF Factor,features
0,0.000000,store_nbr
1,1.180908,preciptotal
2,1.231301,resultdir
3,1.290025,item_nbr
4,1.309410,units
5,1.771422,other
6,2.808411,week7
7,2.834315,weekend
8,4.386178,relative_humility
9,6.097093,resultspeed


In [11]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
model_df20 = sm.OLS.from_formula('np.log1p(units) ~ scale(relative_humility) + C(week7) + C(item_nbr) + 0', data = df20)
result_df20 = model_df20.fit()

print(result_df20.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     2045.
Date:                Thu, 05 Jul 2018   Prob (F-statistic):               0.00
Time:                        21:59:09   Log-Likelihood:                -1463.9
No. Observations:                2627   AIC:                             2950.
Df Residuals:                    2616   BIC:                             3014.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(week7)[0]             

In [12]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
X20 = df20[['relative_humility', 'week7', 'item_nbr']]
y20 = df20['units']
model20 = LinearRegression()

cv20 = KFold(n_splits=20, shuffle=True, random_state=0)

cross_val_score(model20, X20, y20, scoring="r2", cv=cv20)

array([0.22571041, 0.03985767, 0.24265146, 0.21109944, 0.084116  ,
       0.25125265, 0.20465613, 0.16426525, 0.17990415, 0.16864275,
       0.30139693, 0.28670517, 0.22131298, 0.27219382, 0.24062743,
       0.21201444, 0.25390479, 0.20380367, 0.14565353, 0.26726465])

In [12]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
X20 = df20[['relative_humility', 'week7', 'item_nbr']]
y20 = df20['units']
model20 = LinearRegression()

cv20 = KFold(n_splits=20, shuffle=False, random_state=0)

cross_val_score(model20, X20, y20, scoring="r2", cv=cv20)

array([ 0.23801863,  0.27126644,  0.24106082,  0.02271855,  0.12840934,
        0.01512015, -0.0804081 ,  0.05882552, -0.01240345, -0.20003239,
        0.32360017,  0.31576601,  0.3170181 ,  0.39991462,  0.44775809,
        0.41510643,  0.39932637,  0.23564558, -0.08004231, -1.39008131])